In [55]:
import socket 
from threading import Thread
from time import localtime, strftime
import Pyro4.core
import Pyro4.naming 
import pyvisa as visa

In [53]:
###########################################################################
#                              Set defaults                               #
###########################################################################

ns_host = '192.168.3.1' #host ip
ns_port = 9090


class VISAServer(Pyro4.core.ObjBase):
   # """
    #VISAServer is the ``Pyro`` published class that provides remote access to
    #NI Visa via Pyvisa.
    #Notes
    #-----
    #We create visa instances of which we keep track of in the `_connectedres`
    #dictionary. It has the form {visa_resource: instance, ...}, where
    #visa_resource is the unique visa resource name.
    #"""
    def __init__(self):
        Pyro4.core.ObjBase.__init__(self)
        self._resmanager = visa.ResourceManager()
        self._connectedres = {}
        self.reportconnected()

    def reportconnected(self):
        """
        print(all connected visa instruments.) 
        """
        print('Visa connected resources:') 
        print('-------------------------') 
        print(self._resmanager.list_resources()) 
        print('-------------------------') 

    def connectiontest(self):
        """Method used by the remote instance to test the connection.
        """
        return True

    def opendev(self, visa_resource):
        """
        Open a visa device and add it to the `_connectedres` dictionary.
        """
        print(strftime("%d %b %Y %H:%M:%S", localtime()) + ': opendev ' + visa_resource) 
        inst = self._resmanager.open_resource(visa_resource)
        print('Opening device %s -- Done' % visa_resource) 
        self._connectedres[visa_resource] = inst

    def close(self, visa_resource):
        """
        Close the resource and remove it from the `_connectedres` dictionary.
        """
        print(strftime("%d %b %Y %H:%M:%S", localtime()) + ': close ' + visa_resource) 
        self._connectedres[visa_resource].close()
        del self._connectedres[visa_resource]

    def set_timeout(self, visa_resource, val):
        self._connectedres[visa_resource].timeout = val

    def ask(self, visa_resource, question):
        print(strftime("%d %b %Y %H:%M:%S", localtime()) + ': ask ' + visa_resource + ' for ' + question) 
        res = self._connectedres[visa_resource].ask(question)
        print(res) 
        print(type(res)) 
        return res

    def write(self, visa_resource, command):
        print(strftime("%d %b %Y %H:%M:%S", localtime()) + ': write ' + visa_resource + ' text ' + command) 
        self._connectedres[visa_resource].write(command)

    def read(self, visa_resource):
        print(strftime("%d %b %Y %H:%M:%S", localtime()) + ': read ' + visa_resource) 
        res = self._connectedres[visa_resource].read()
        return res


class VISAServerDaemon(Thread):
    """
    The server thread that hosts the ``Pyro`` daemon.
    """
    def __init__(self, daemon):
        Thread.__init__(self)
        self.pyroDaemon = daemon
        self.setDaemon(True)

    def run(self):
        try:
            self.pyroDaemon.requestLoop()
        finally:
            print('shutting down') 
            self.pyroDaemon.shutdown(True)



def getHostIP():
    """
    Return the host ip of the computer.
    """
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        myip = s.getsockname()[0]
        s.close()
    except:
        myip = socket.gethostbyname(socket.gethostname())
        print("failed to get ip by connecting to external server. using %s" % myip) 
    return myip


def main():
    Pyro.config.PYRO_NS_HOSTNAME = ns_host
    Pyro.config.PYRO_NS_PORT = ns_port
    myIP = getHostIP()
    Pyro.config.PYRO_HOST = myIP
    # connect to an existing nameserver
    ns = Pyro.naming.NameServerLocator().getNS()
    print( 'name server found at %s' % ns.URI)
    Pyro4.core.initServer()
    daemon = Pyro4.core.Daemon()
    daemon.useNameServer(ns)
    visaserver = VISAServer()
    print('VISA server running on %s')  % \
        daemon.connectPersistent(visaserver, 'VISAServer')
    visadaemon = VISAServerDaemon(daemon)
    visadaemon.start()
    return visaserver


if __name__ == '__main__':
    visaserver = main()
    
# 2022 GitHub, Inc.
#Terms
#Privacy
#Security
#Status
#Docs
#Contact GitHub
#Pri

AttributeError: 'Configuration' object has no attribute 'PYRO_NS_HOSTNAME'

In [11]:
class Person:
  def __init__(n ,name, age):
    n.name = name
    n.age = age

p1 = Person("John", 36)

print(p1.name)
print(p1.age)

John
36


In [ ]:
class GreetingMaker(object):
    def get_fortune(self, name):
        return "Hello, {0}. Here is your fortune message:\n" 

uri = Pyro4.Daemon().register(GreetingMaker) #register GreetingMaker as pyro object and a daemon
print("Ready object = " + str(uri))

print("Ready.")
daemon.requestLoop()




In [ ]:
greeting_maker = Pyro4.Proxy("PYRONAME:example.greeting")    # use name server object lookup uri shortcut
print(greeting_maker.get_fortune(name))